# ================================================================
# 🧠 DENSE vs ACTIVE PARAMETER MODEL COMPARISON
# ================================================================
# This notebook compares text generation outputs from two LLMs:
# 1️⃣ Mistral (Dense Model)
# 2️⃣ Mixtral (Mixture-of-Experts or Active Parameter Model)
#
# Instead of loading models locally (which is very resource heavy),
# we use Ollama to query the models through its local inference API.
# Make sure Ollama is installed and running on your system.
#
# You can check running models with:
#   ollama list
# and pull models with:
#   ollama pull mistral
#   ollama pull mixtral
#
# Update the base_url if your Ollama server runs on a remote host.
# ================================================================

from langchain_community.chat_models import ChatOllama

# Define the Ollama base URL (change if running remotely)
OLLAMA_BASE_URL = "http://10.79.253.112:11434"

print("✅ Ollama base URL set to:", OLLAMA_BASE_URL)


## Part 0 — Setup & Installation Notes

Run the cell below to install required packages if you haven't already.


In [ ]:
!pip install -U pip
!pip install -q "transformers>=4.35.0" accelerate datasets evaluate sentencepiece
!pip install huggingface_hub




## Part 0.1 — Environment Check (device, torch, memory)

This cell detects available device (cuda/mps/cpu), PyTorch version, and prints basic info. On PCAI you'll see `cuda` device;

In [ ]:
import torch, platform, gc
print('Python:', platform.python_version())
print('PyTorch:', torch.__version__)
device = None
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Using CUDA device:', torch.cuda.get_device_name(0))


# helper to show memory info (torch.cuda only)
def print_mem():
    if device.type == 'cuda':
        torch.cuda.synchronize()
        print('Allocated (MB):', torch.cuda.memory_allocated()/1e6)
        print('Reserved (MB):', torch.cuda.memory_reserved()/1e6)

print_mem()

Python: 3.12.12
PyTorch: 2.8.0+cu126
Using CUDA device: Tesla T4
Allocated (MB): 0.0
Reserved (MB): 0.0


## Part 1 — Model selection



In [ ]:
# Model IDs (Hugging Face / public model hub ids)

MISTRAL_DENSE_ID = "mistralai/Mistral-7B-Instruct-v0.2"     # dense 7B
MIXTRAL_MOE_ID   = "mistralai/Mixtral-8x7B-Instruct-v0.1"   # Mixtral MoE 8x7B (may be large)



## Part 2 — Utilities: tokenization, generation helper, timing

We define helper functions to generate text, measure latency, and display outputs cleanly. The generation function uses `model.generate` from `transformers`.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import time
from IPython.display import Markdown, display

# 🔹 Function to create a text generator for a given model and tokenizer
def make_generator(model, tokenizer, device):
    def generate_text(prompt, max_new_tokens=200, temperature=0.7, top_p=0.9):
        inputs = tokenizer(prompt, return_tensors='pt', truncation=True, padding=True).to(device)
        t0 = time.time()
        out_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )
        t1 = time.time()
        text = tokenizer.decode(out_ids[0], skip_special_tokens=True)
        return text, t1 - t0
    return generate_text



## Part 3 — Load models

This cell tries to load the primary Mistral/Mixtral models when running on a CUDA GPU with enough memory.

In [ ]:
import traceback
from transformers import AutoTokenizer, AutoModelForCausalLM

# --- Model containers ---
models = {'dense': None, 'moe': None}
tokenizers = {'dense': None, 'moe': None}
loaded_real_mixtral = False

# --- Model IDs ---
MISTRAL_DENSE_ID = "mistralai/Mistral-7B-v0.1"
MIXTRAL_MOE_ID = "mistralai/Mixtral-8x7B-v0.1"

# --- Load models ---
try:
    print('Attempting to load Mistral-7B dense model...')
    tok_dense = AutoTokenizer.from_pretrained(MISTRAL_DENSE_ID)
    mdl_dense = AutoModelForCausalLM.from_pretrained(
        MISTRAL_DENSE_ID, torch_dtype="auto", device_map="auto"
    )
    models['dense'] = mdl_dense
    tokenizers['dense'] = tok_dense
    print('✅ Loaded dense Mistral-7B.')
except Exception as e:
    print('❌ Failed to load Mistral-7B dense:', e)
    traceback.print_exc()

try:
    print('Attempting to load Mixtral-8x7B MoE model...')
    tok_moe = AutoTokenizer.from_pretrained(MIXTRAL_MOE_ID)
    mdl_moe = AutoModelForCausalLM.from_pretrained(
        MIXTRAL_MOE_ID, torch_dtype="auto", device_map="auto"
    )
    models['moe'] = mdl_moe
    tokenizers['moe'] = tok_moe
    loaded_real_mixtral = True
    print('✅ Loaded Mixtral-8x7B MoE.')
except Exception as e:
    print('❌ Failed to load Mixtral-8x7B:', e)
    traceback.print_exc()

# --- Fix pad_token if missing ---
for name, tok in tokenizers.items():
    if tok is not None and tok.pad_token is None:
        tok.pad_token = tok.eos_token
        print(f"Assigned pad_token for {name} tokenizer to eos_token.")

# --- Summary ---
print('\nModel load summary:')
for k in models:
    print(
        f"{k}:",
        "✅ real Mixtral" if (k == 'moe' and loaded_real_mixtral)
        else "✅ loaded" if models[k]
        else "⚠️ none"
    )

loaded_real_mixtral = False

# Try loading the ideal pair on CUDA (PCAI)
if torch.cuda.is_available():
    try:
        print('Attempting to load Mistral-7B dense model...')
        tok_dense = AutoTokenizer.from_pretrained(MISTRAL_DENSE_ID)
        mdl_dense = AutoModelForCausalLM.from_pretrained(MISTRAL_DENSE_ID, torch_dtype=torch.float16, device_map='auto')
        models['dense'] = mdl_dense
        tokenizers['dense'] = tok_dense
        print('Loaded dense Mistral-7B.')
    except Exception as e:
        print('Failed to load Mistral-7B dense:', e)
        traceback.print_exc()

    try:
        print('Attempting to load Mixtral-8x7B MoE model...')
        tok_moe = AutoTokenizer.from_pretrained(MIXTRAL_MOE_ID)
        mdl_moe = AutoModelForCausalLM.from_pretrained(MIXTRAL_MOE_ID, torch_dtype=torch.float16, device_map='auto')
        models['moe'] = mdl_moe
        tokenizers['moe'] = tok_moe
        loaded_real_mixtral = True
        print('Loaded Mixtral MoE.')
    except Exception as e:
        print('Failed to load Mixtral-8x7B:', e)
        traceback.print_exc()



# Ensure all tokenizers have a valid pad_token
for name, tok in tokenizers.items():
    if tok is not None and tok.pad_token is None:
        tok.pad_token = tok.eos_token
        print(f"Assigned pad_token for {name} tokenizer to eos_token.")

print('\nModel load summary:')
for k in models:
    print(k, '->', 'real Mixtral' if (k=='moe' and loaded_real_mixtral)
          else ('simulated' if k=='moe' else 'loaded' if models[k] else 'none'))

Attempting to load Mistral-7B dense model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Loaded dense Mistral-7B.
Attempting to load Mixtral-8x7B MoE model...


Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

...

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:798: UserWarning: Not enough free disk space to download the file. The expected file size is: 4221.68 MB. The target location /root/.cache/huggingface/hub/models--mistralai--Mixtral-8x7B-v0.1/blobs only has 0.00 MB free disk space.
  warnings.warn(


model-00019-of-00019.safetensors:   0%|          | 0.00/4.22G [00:00<?, ?B/s]

❌ Failed to load Mixtral-8x7B: Data processing error: CAS service error : IO Error: No space left on device (os error 28)
Assigned pad_token for dense tokenizer to eos_token.

Model load summary:
dense: ✅ loaded
moe: ⚠️ none
Attempting to load Mistral-7B dense model...


Traceback (most recent call last):
  File "/tmp/ipython-input-1325143883.py", line 30, in <cell line: 0>
    mdl_moe = AutoModelForCausalLM.from_pretrained(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py", line 604, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py", line 277, in _wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py", line 4900, in from_pretrained
    checkpoint_files, sharded_metadata = _get_resolved_checkpoint_files(
                                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py", line 1200, in _get_resolved_checkpoint_files
    checkpoint_files, sharded_metadata 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Part 4 — Tasks: Content Writing, Code Generation, Summarization

Below we define three prompts. We'll generate outputs from both the dense model and the MoE.


In [ ]:
# Prompts
prompt_content = """Write a 3-paragraph blog post (with headings) about the impact of Mixture-of-Experts (MoE) architectures "
on modern large language models, focusing on why active-parameter scaling matters and one practical use-case."""

prompt_code = """Write a clear Python function named `count_primes(n)` that returns the number of prime numbers less than or equal to n.
Include docstring, type hints, and a brief complexity analysis comment."""

prompt_summary = ("""Summarize the following paragraph in 2 short sentences:
                   HPE ProLiant servers—The world’s most secure industry standard servers. HPE ProLiant Gen10 and Gen10 Plus servers
                   paired with HPE OneView and HPE InfoSight deliver software-defined compute to accelerate application performance and improve server operations.
                   Their wide selection of multicore, multiprocessor servers and server blades meet needs from small businesses to global enterprises.""")

In [ ]:
#Generate output for Mistral (Dense Model)
compute_device = torch.device("cuda")

dense_model = models['dense']
dense_gen = make_generator(dense_model, tokenizers['dense'], compute_device)

print(f"Running on device: {compute_device}")
print('Generating content using **Mistral (Dense Model)**...\n')

dense_out, dense_time = dense_gen(prompt_content, max_new_tokens=250, temperature=0.8)

print("=== Mistral (Dense Model) Output ===\n")
print(dense_out)
print(f"\n⏱️  Generation Time: {dense_time:.3f} s")


In [ ]:
#Generate output for Mixtral (Active Parameter / MoE Model)
compute_device = torch.device("cuda")

moe_model = models['moe'].base if hasattr(models['moe'], 'base') else models['moe']
moe_gen = make_generator(moe_model, tokenizers['moe'], compute_device)

print(f"Running on device: {compute_device}")
print('Generating content using **Mixtral (Active Parameter Model)**...\n')

moe_out, moe_time = moe_gen(prompt_content, max_new_tokens=250, temperature=0.8)

print("=== Mixtral (Active Parameter Model) Output ===\n")
print(moe_out)
print(f"\n⚙️  Generation Time: {moe_time:.3f} s")

In [ ]:
# Code generation
print('Generating code (dense)...')
dense_code, dt = dense_gen(prompt_code, max_new_tokens=200, temperature=0.2, top_p=0.95)
print('Generating code (moe)...')
moe_code, mt = moe_gen(prompt_code, max_new_tokens=200, temperature=0.2, top_p=0.95)


print(f'Dense time: {dt:.3f}s, MoE time: {mt:.3f}s')

In [ ]:
# Summarization
print('Generating summary (dense)...')
dense_sum, dt2 = dense_gen(prompt_summary, max_new_tokens=120, temperature=0.3)
print('Generating summary (moe)...')
moe_sum, mt2 = moe_gen(prompt_summary, max_new_tokens=120, temperature=0.3)

print(f'Dense time: {dt2:.3f}s, MoE time: {mt2:.3f}s')

## Part 5 — Simple Quantitative Comparison (latency & token length)

We compute generation times and output token lengths for each task to compare speed and verbosity. This is a lightweight, approximate comparison — for rigorous benchmarking run multiple iterations and average.


In [ ]:
def output_stats(text, elapsed):
    tokens = len(text.split())
    return {'tokens': tokens, 'time_s': elapsed, 'tokens_per_s': tokens / elapsed if elapsed>0 else 0}

stats = {
    'content_dense': output_stats(dense_out, dense_time),
    'content_moe': output_stats(moe_out, moe_time),
    'code_dense': output_stats(dense_code, dt),
    'code_moe': output_stats(moe_code, mt),
    'sum_dense': output_stats(dense_sum, dt2),
    'sum_moe': output_stats(moe_sum, mt2),
}

import pandas as pd
df = pd.DataFrame.from_dict(stats, orient='index')
display(df.style.format({'time_s':'{:.3f}','tokens_per_s':'{:.1f}'}))

## Part 6 — Notes on Active Parameters & Model Introspection

If you loaded the real Mixtral MoE model, some libraries expose routing / expert usage stats. For Hugging Face transformers, MoE models may expose `model.config` entries or custom attributes. If you used a simulated MoE, the notebook cannot report real active parameter counts; instead it demonstrates qualitative differences.

If you want, we can add a cell that attempts to read `model.config` and prints any MoE-related fields (e.g., `moe_num_experts`, `moe_k`, `expert_parallelism`).


In [ ]:
# Inspect config for Moe-specific info (if available)
def inspect_moe_info(model):
    info = {}
    try:
        cfg = model.config
        for k in ['moe_num_experts', 'num_experts', 'moe_k', 'expert_parallelism']:
            if hasattr(cfg, k):
                info[k] = getattr(cfg, k)
        # also dump some basic config fields
        info['hidden_size'] = getattr(cfg, 'hidden_size', None)
        info['n_layer'] = getattr(cfg, 'n_layer', None)
    except Exception as e:
        info['error'] = str(e)
    return info

print('Dense model config snippet:', inspect_moe_info(models['dense']))
print('MoE model config snippet:', inspect_moe_info(models['moe'].base if hasattr(models['moe'], 'base') else models['moe']))